In [113]:
%matplotlib inline

import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

In [114]:
pd. set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 0)

In [115]:
alphavantage_csv_path = "/app/StockPricePredictions/data/alphavantage/time_series_daily_adjusted/AAPL.csv"

In [116]:
df = pd.read_csv(alphavantage_csv_path, low_memory=False)
df.sort_values(by=["date"], ascending=True, inplace=True)

df = df[df["date"]>='2010-01-01']

df["date"] = pd.to_datetime(df['date'])

df.set_index("date", inplace=True)

In [117]:
df.shape[0]

3064

In [118]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3064 entries, 2010-01-04 to 2022-03-04
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   1. open               3064 non-null   float64
 1   2. high               3064 non-null   float64
 2   3. low                3064 non-null   float64
 3   4. close              3064 non-null   float64
 4   5. adjusted close     3064 non-null   float64
 5   6. volume             3064 non-null   float64
 6   7. dividend amount    3064 non-null   float64
 7   8. split coefficient  3064 non-null   float64
dtypes: float64(8)
memory usage: 215.4 KB


In [119]:
df.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient
date,,,,,,,,
2010-01-04,213.43,214.50,212.38,214.01,6.543876,17633200.0,0.0,1.0
2010-01-05,214.60,215.59,213.25,214.38,6.555190,21496600.0,0.0,1.0
2010-01-06,214.38,215.23,210.75,210.97,6.450921,19720000.0,0.0,1.0
2010-01-07,211.75,212.00,209.05,210.58,6.438996,17040400.0,0.0,1.0
2010-01-08,210.30,212.00,209.06,211.98,6.481804,15986100.0,0.0,1.0


In [120]:
benzinga_csv_path = "/app/StockPricePredictions/data/benzinga/aapl_non_dupes.csv"

In [121]:
df_benzinga = pd.read_csv(benzinga_csv_path, low_memory=False)

In [122]:
df_benzinga['date'] = pd.to_datetime(df_benzinga['date'])

In [123]:
df_benzinga.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29077 entries, 0 to 29076
Data columns (total 19 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      29077 non-null  datetime64[ns]
 1   author    29077 non-null  object        
 2   created   29077 non-null  object        
 3   updated   29077 non-null  object        
 4   title     29077 non-null  object        
 5   teaser    4824 non-null   object        
 6   body      23916 non-null  object        
 7   url       29077 non-null  object        
 8   image     29077 non-null  object        
 9   channels  29077 non-null  object        
 10  stocks    29077 non-null  object        
 11  tags      29077 non-null  object        
 12  AAPL      29077 non-null  bool          
 13  MSFT      29077 non-null  bool          
 14  AMZN      29077 non-null  bool          
 15  FB        29077 non-null  bool          
 16  NVDA      29077 non-null  bool          
 17  INTC      29

In [124]:
df_benzinga.columns

Index(['date', 'author', 'created', 'updated', 'title', 'teaser', 'body',
       'url', 'image', 'channels', 'stocks', 'tags', 'AAPL', 'MSFT', 'AMZN',
       'FB', 'NVDA', 'INTC', 'NFLX'],
      dtype='object')

In [125]:
df_benzinga["text"] = df_benzinga["title"] + " " + df_benzinga["body"]

In [126]:
df_benzinga['text'] = np.where(df_benzinga["text"], df_benzinga["title"], df_benzinga["text"])

In [127]:
df_benzinga = df_benzinga[[ "date", "text"]]

In [128]:
df_benzinga.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29077 entries, 0 to 29076
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    29077 non-null  datetime64[ns]
 1   text    29077 non-null  object        
dtypes: datetime64[ns](1), object(1)
memory usage: 454.5+ KB


In [129]:
df_benzinga.head()

,date,text
0,2010-01-02,Time to Sell Apple Puts
1,2010-01-04,"Nexus – Will It Change Telecom Industry Dynamics? (GOOG, AAPL, RIMM, ATT, VZ)"
2,2010-01-04,"Company News for January 04, 2010 - Corporate Summary"
3,2010-01-04,"Technology Industry Update (DELL, AAPL, AMD, INTC, NVD)"
4,2010-01-04,Apple Defies Gravity


In [130]:
def clean_tweet(df):
    # replace URLs with a whitespace
    df['text'] = df['text'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ')
    
    return df

In [131]:
df_benzinga = clean_tweet(df_benzinga)

<ipython-input-130-69350fdf1d4b>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ')


In [132]:
df_benzinga.head()

,date,text
0,2010-01-02,Time to Sell Apple Puts
1,2010-01-04,"Nexus – Will It Change Telecom Industry Dynamics? (GOOG, AAPL, RIMM, ATT, VZ)"
2,2010-01-04,"Company News for January 04, 2010 - Corporate Summary"
3,2010-01-04,"Technology Industry Update (DELL, AAPL, AMD, INTC, NVD)"
4,2010-01-04,Apple Defies Gravity


In [133]:
idx = 0

### Vader Sentiment

In [134]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()
sentence = df_benzinga["text"][idx] 
vs = analyzer.polarity_scores(sentence)
print("{:-<65} {}".format(sentence, str(vs)))

Time to Sell Apple Puts------------------------------------------ {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


In [135]:
def getSIA(text):
    sia = SentimentIntensityAnalyzer()
    sentiment = sia.polarity_scores(text)
    return sentiment

negative = []
neutral = []
positive = []
compound = []

for i in range(0, len(df_benzinga["text"])):
    sia = getSIA(df_benzinga["text"][i])
    negative.append(sia["neg"])
    neutral.append(sia["neu"])
    positive.append(sia["pos"])
    compound.append(sia["compound"])

df_benzinga["negative"] = negative
df_benzinga["neutral"] = neutral
df_benzinga["positive"] = positive
df_benzinga["compound"] = compound

In [136]:
df_benzinga.set_index("date", inplace=True)

In [137]:
df_benzinga.head()

,text,negative,neutral,positive,compound
date,,,,,
2010-01-02,Time to Sell Apple Puts,0.0,1.000,0.000,0.0000
2010-01-04,"Nexus – Will It Change Telecom Industry Dynamics? (GOOG, AAPL, RIMM, ATT, VZ)",0.0,0.851,0.149,0.2732
2010-01-04,"Company News for January 04, 2010 - Corporate Summary",0.0,1.000,0.000,0.0000
2010-01-04,"Technology Industry Update (DELL, AAPL, AMD, INTC, NVD)",0.0,1.000,0.000,0.0000
2010-01-04,Apple Defies Gravity,0.0,1.000,0.000,0.0000


In [138]:
df_benzinga_duplicated_index = df_benzinga[df_benzinga.index.duplicated(keep=False)]

In [139]:
df_benzinga_duplicated_index.shape[0]

28562

In [140]:
df_benzinga_nonduplicated_index = df_benzinga[~df_benzinga.index.duplicated(keep=False)]

In [141]:
df_benzinga_nonduplicated_index.shape[0]

515

In [142]:
# df_twint.groupby('date')['negative'].mean()

df_benzinga_groupby = df_benzinga.groupby("date").agg(
     negative = ("negative", "mean"),
     nuetral = ("neutral", "mean"),
     positive = ("positive", "mean"),
     compound = ("compound", "mean"),
     # nlikes = ("nlikes", "sum"),
     # nretweets = ("nretweets", "sum"),
     )


In [143]:
df_benzinga_groupby

,negative,nuetral,positive,compound
date,,,,
2010-01-02,0.000000,1.000000,0.000000,0.000000
2010-01-04,0.023300,0.946500,0.030200,0.012850
2010-01-05,0.039714,0.917929,0.042357,0.019529
2010-01-06,0.000000,0.929800,0.070200,0.090833
2010-01-07,0.000000,0.862250,0.137750,0.279775
...,...,...,...,...
2022-02-28,0.090125,0.785000,0.124875,-0.053988
2022-03-01,0.104500,0.841750,0.053750,-0.057088
2022-03-02,0.039500,0.894333,0.066167,0.014058


In [144]:
df_merge = pd.merge(df, df_benzinga_groupby, how="left", left_index=True, right_index=True)

In [145]:
# df.index
# df_text.index
df_merge[df_merge.compound.isnull()]
# df_merge.shape

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,negative,nuetral,positive,compound
date,,,,,,,,,,,,
2012-08-10,618.71,621.76,618.7000,621.70,19.091146,6962100.0,0.00,1.0,NaN,NaN,NaN,NaN
2014-07-31,97.16,97.45,95.3300,95.60,21.409463,56843000.0,0.00,1.0,NaN,NaN,NaN,NaN
2016-06-22,96.25,96.89,95.3500,95.55,22.217966,29219122.0,0.00,1.0,NaN,NaN,NaN,NaN
2016-11-04,108.53,110.25,108.1100,108.84,25.576562,30836997.0,0.00,1.0,NaN,NaN,NaN,NaN
2016-12-23,115.59,116.52,115.5900,116.52,27.381303,14249484.0,0.00,1.0,NaN,NaN,NaN,NaN
2017-11-24,175.10,175.50,174.6459,174.97,41.780905,14026519.0,0.00,1.0,NaN,NaN,NaN,NaN
2018-02-23,173.67,175.65,173.5400,175.50,42.076261,33329232.0,0.00,1.0,NaN,NaN,NaN,NaN
2018-03-02,172.80,176.30,172.4500,176.21,42.246484,38453950.0,0.00,1.0,NaN,NaN,NaN,NaN
2018-03-09,177.96,180.00,177.3900,179.98,43.150344,31385134.0,0.00,1.0,NaN,NaN,NaN,NaN


In [109]:
df.shape

(3064, 8)

In [146]:
df_benzinga_groupby.shape

(3846, 4)

In [147]:
df_merge.fillna(method="ffill", inplace=True)

In [149]:
df_merge[df_merge.compound.isnull()]

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,negative,nuetral,positive,compound
date,,,,,,,,,,,,


In [151]:
# df_merge.to_csv("/app/StockPricePredictions/data/alphavantage/time_series_daily_adjusted/AAPL_WITH_BENZINGA_SA.csv")
df_merge.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,negative,nuetral,positive,compound
date,,,,,,,,,,,,
2010-01-04,213.43,214.50,212.38,214.01,6.543876,17633200.0,0.0,1.0,0.023300,0.946500,0.030200,0.012850
2010-01-05,214.60,215.59,213.25,214.38,6.555190,21496600.0,0.0,1.0,0.039714,0.917929,0.042357,0.019529
2010-01-06,214.38,215.23,210.75,210.97,6.450921,19720000.0,0.0,1.0,0.000000,0.929800,0.070200,0.090833
2010-01-07,211.75,212.00,209.05,210.58,6.438996,17040400.0,0.0,1.0,0.000000,0.862250,0.137750,0.279775
2010-01-08,210.30,212.00,209.06,211.98,6.481804,15986100.0,0.0,1.0,0.016000,0.910100,0.073900,0.070060


# Spacy TextBlob

In [119]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

nlp = spacy.load('en_core_web_md')
nlp.add_pipe('spacytextblob')
text = 'Nexus – Will It Change Telecom Industry Dynamics? '
doc = nlp(text)

In [120]:
print(doc._.blob.polarity)
# -0.125

print(doc._.blob.subjectivity)
# 0.9

0.0
0.0


# TextBlob

In [122]:
from textblob import TextBlob

def sentiment_analysis(tweet):

 def getSubjectivity(text):
   return TextBlob(text).sentiment.subjectivity
  
 #Create a function to get the polarity
 def getPolarity(text):
   return TextBlob(text).sentiment.polarity
  
 #Create two new columns ‘Subjectivity’ & ‘Polarity’

 subjectivity = getSubjectivity(tweet)
 polarity = getPolarity(tweet)

 def getAnalysis(score):
  if score < 0:
    return "Negative"
  elif score == 0:
    return "Neutral"
  else:
    return "Positive"

 # tweet [‘TextBlob_Analysis’] = tweet  [‘TextBlob_Polarity’].apply(getAnalysis )

 result = getAnalysis(polarity)

 print('Subjectivity:', subjectivity)
 print('Polarity:', polarity)
 print('Analysis:', result)
 # print('Sentiment:', sentiment)

idx = 5

sentiment_analysis(df_twint["tweet"][idx])

df_twint["tweet"][idx]

Subjectivity: 0.0
Polarity: 0.0
Analysis: Neutral


'Why Apple Bought Quattro Wireless And Is Getting Into Advertising $AAPL $GOOG by @fromedome   '